## 1. Resize Images and Adjust Bounding Boxes


1.1 Resize images to YOLO requered ar/resolution (416x416)

In [3]:
import cv2
import os

image_directory = '../lens_images_v2.0/backup/'
output_directory = './resized_images/'

if not os.path.exists(output_directory):
    os.mkdir(output_directory)

for filename in os.listdir(image_directory):
    if filename.endswith('.jpeg'):
        image_path = os.path.join(image_directory, filename)
        image = cv2.imread(image_path)

        # Determine the crop dimensions (cropping to 3024x3024)
        crop_size = 3024
        start_x = (image.shape[1] - crop_size) // 2
        start_y = (image.shape[0] - crop_size) // 2
        cropped_image = image[start_y:start_y+crop_size, start_x:start_x+crop_size]

        # Resize the cropped image to 416x416
        resized_image = cv2.resize(cropped_image, (416, 416))
        output_path = os.path.join(output_directory, filename)
        cv2.imwrite(output_path, resized_image)


1.2 Adjust bounding boxes

In [36]:
def adjust_bounding_boxes(input_path, output_path):
    # Define the scaling factor for the y direction (height)
    scaling_factor_y = 1.33333333333

    # Read bounding boxes from the YOLO txt file
    with open(input_path, 'r') as file:
        boxes = [list(map(float, line.strip().split())) for line in file.readlines()]

    # Adjust the height of the bounding boxes
    adjusted_boxes = []
    for box in boxes:
        class_id, x_center, y_center, width, height = box
        adjusted_height = height * scaling_factor_y
        adjusted_box = (
            class_id,
            x_center,
            y_center,
            width,
            adjusted_height
        )
        adjusted_boxes.append(adjusted_box)

    # Write the adjusted bounding boxes to the new txt file
    with open(output_path, 'w') as file:
        for box in adjusted_boxes:
            file.write(' '.join(map(str, box)) + '\n')


In [40]:
original_size = (3024, 4032)
target_size = (416, 416)
crop_size = 3024

text_directory = 'yolo_txt'
output_text_directory = 'resized_yolo_txt'


if not os.path.exists('resized_yolo_txt'):
    os.mkdir('resized_yolo_txt')

for filename in os.listdir(text_directory):
    if filename.endswith('.txt'):
        input_path = os.path.join(text_directory, filename)
        output_path = os.path.join(output_text_directory, filename)
        adjust_bounding_boxes(input_path, output_path)

## 3. Perform image augmentation 

- HorizontalFlip(p=0.5)
- VerticalFlip(p=0.5)
- Rotate(limit=30, p=0.5), **removed**
- RandomBrightnessContrast(p=0.2)

In [6]:
import os
import cv2
import numpy as np
from albumentations import (
    Compose, HorizontalFlip, VerticalFlip, RandomBrightnessContrast, Rotate
)

def augment_images_and_annotations(image_dir, annotations_dir, output_image_dir, output_annotation_dir, augmentations=4):
    # Augmentation pipeline
    transforms = Compose([
        HorizontalFlip(p=0.5), # Horizontal Flip
        VerticalFlip(p=0.5),   # Vertical Flip
        #Rotate(limit=30, p=0.5), # Rotation up to 30 degrees
        RandomBrightnessContrast(p=0.2), # Random brightness and contrast adjustments
    ], bbox_params={'format': 'yolo', 'label_fields': ['class_labels']})

    # List image files
    for image_file in os.listdir(image_dir):
        if not image_file.endswith('.png') :
            continue

        # Corresponding annotation file
        annotation_file = image_file.replace('.png', '.txt')

        # Read image
        image_path = os.path.join(image_dir, image_file)
        image = cv2.imread(image_path)

        # Save original image
        image_output_path = os.path.join(output_image_dir, image_file)
        cv2.imwrite(image_output_path, image)

        # Read bounding boxes
        annotation_path = os.path.join(annotations_dir, annotation_file)
        bounding_boxes = []
        class_labels = []
        with open(annotation_path, 'r') as file:
            for line in file.readlines():
                class_id, x_center, y_center, width, height = map(float, line.strip().split())
                bounding_boxes.append([x_center, y_center, width, height])
                class_labels.append(int(class_id))

        # Save original annotations
        annotation_output_path = os.path.join(output_annotation_dir, annotation_file)
        with open(annotation_output_path, 'w') as file:
            for bb, class_id in zip(bounding_boxes, class_labels):
                file.write(f"{class_id} {' '.join(map(str, bb))}\n")

        # Perform augmentations
        for i in range(augmentations):
            augmented = transforms(image=image, bboxes=bounding_boxes, class_labels=class_labels)
            image_aug = augmented['image']
            bboxes_aug = augmented['bboxes']

            # Save augmented image
            image_filename = image_file.replace('.png', f'_aug{i}.png')
            image_output_path = os.path.join(output_image_dir, image_filename)
            cv2.imwrite(image_output_path, image_aug)

            # Save augmented annotations
            annotation_filename = annotation_file.replace('.txt', f'_aug{i}.txt')
            annotation_output_path = os.path.join(output_annotation_dir, annotation_filename)
            with open(annotation_output_path, 'w') as file:
                for bb, class_id in zip(bboxes_aug, augmented['class_labels']):
                    file.write(f"{class_id} {' '.join(map(str, bb))}\n")

In [57]:
image_dir = 'resized_images'
annotations_dir = 'resized_yolo_txt'
output_image_dir = 'augmented_images'
output_annotation_dir = 'augmented_yolo_txt'

augment_images_and_annotations(image_dir, annotations_dir, output_image_dir, output_annotation_dir)

## 3. Split data into train/validate/test

80/10/10

In [59]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Paths to your original images and annotations
image_directory = 'augmented_images'
annotation_directory = 'augmented_yolo_txt'

# Create main directory
main_directory = 'lens_dataset_aug2023'
os.makedirs(main_directory, exist_ok=True)

# Create subdirectories for training, validation, and testing
training_dir = os.path.join(main_directory, 'training')
validation_dir = os.path.join(main_directory, 'validation')
testing_dir = os.path.join(main_directory, 'testing')

for dir in [training_dir, validation_dir, testing_dir]:
    os.makedirs(os.path.join(dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(dir, 'annotations'), exist_ok=True)

# Get list of image and annotation files
image_files = [f for f in os.listdir(image_directory) if f.endswith('.jpeg')]
annotation_files = [f for f in os.listdir(annotation_directory) if f.endswith('.txt')]

# Split into training, validation, and testing
train_files, test_files = train_test_split(image_files, test_size=0.20, random_state=42)
val_files, test_files = train_test_split(test_files, test_size=0.50, random_state=42)

# Function to copy files
def copy_files(files, src_image_dir, src_anno_dir, dest_image_dir, dest_anno_dir):
    for filename in files:
        shutil.copy(os.path.join(src_image_dir, filename), os.path.join(dest_image_dir, filename))
        annotation_file = filename.replace('.jpeg', '.txt')
        shutil.copy(os.path.join(src_anno_dir, annotation_file), os.path.join(dest_anno_dir, annotation_file))

# Copy files to respective directories
copy_files(train_files, image_directory, annotation_directory, os.path.join(training_dir, 'images'), os.path.join(training_dir, 'annotations'))
copy_files(val_files, image_directory, annotation_directory, os.path.join(validation_dir, 'images'), os.path.join(validation_dir, 'annotations'))
copy_files(test_files, image_directory, annotation_directory, os.path.join(testing_dir, 'images'), os.path.join(testing_dir, 'annotations'))

print("Dataset organized successfully!")


Dataset organized successfully!


## 4. Load Pretrained Model and Train

pretrained model `YOLOv8s` downloaded  [here](https://docs.ultralytics.com/models/yolov8/#supported-modes)

In [62]:
from ultralytics import YOLO

# Load a pretrained model (recommended for training)
model = YOLO('yolov8s.pt')

# train the model
results = model.train(data='lens_dataset.yaml', epochs=100, imgsz=416, device='mps')



Ultralytics YOLOv8.0.154 🚀 Python-3.9.6 torch-2.0.1 MPS (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=lens_dataset.yaml, epochs=100, patience=50, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=

KeyboardInterrupt: 

paused training so i could use my computer. resuming ...

In [2]:
from ultralytics import YOLO

# Load the saved model
model = YOLO('/Users/tylerhouchin/runs/detect/train2/weights/last.pt')

# Continue training with the same parameters
results = model.train(data='lens_dataset.yaml', epochs=100, imgsz=416, resume=True)

/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
New https://pypi.org/project/ultralytics/8.0.155 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.154 🚀 Python-3.9.6 torch-2.0.1 MPS (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=/Users/tylerhouchin/runs/detect/train2/weights/last.pt, data=lens_dataset.yaml, epochs=100, patience=50, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.

# 6. Retrain

### 6.1 Extract just the lens bounding boxes from the new dataset

In [1]:
import os

def extract_lens_coordinates(src_dir, dest_dir):
    # Ensure destination directory exists or create it
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # List all txt files in the source directory
    annotation_files = [f for f in os.listdir(src_dir) if f.endswith('.txt')]

    for file in annotation_files:
        # Read the source file
        with open(os.path.join(src_dir, file), 'r') as src_file:
            lines = src_file.readlines()
            
            lens_data = []
            for line in lines:
                # Check if class is 0 (lens)
                if line.startswith('0'):
                    lens_data.append(line)
            
            # Write the lens data to the destination file
            with open(os.path.join(dest_dir, file), 'w') as dest_file:
                dest_file.writelines(lens_data)


In [2]:
src_directory = "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/yolo/capture_data/2023-08-22/txt"
dest_directory = "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/yolo/capture_data/2023-08-22/txt/just_lens"
extract_lens_coordinates(src_directory, dest_directory)

### 6.2 Augment images & annotations

In [7]:
image_dir = '/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/yolo/capture_data/2023-08-22/images/'
annotations_dir = '/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/yolo/capture_data/2023-08-22/txt/just_lens/'
output_image_dir = '/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/yolo/capture_data/2023-08-22/augmented/images/'
output_annotation_dir = '/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/yolo/capture_data/2023-08-22/augmented/txt/just_lens/'

augment_images_and_annotations(image_dir, annotations_dir, output_image_dir, output_annotation_dir)

### 6.3 Split into Train/Test/Val

In [8]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Paths to your original images and annotations
image_directory = '/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/yolo/capture_data/2023-08-22/augmented/images/'
annotation_directory = '/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/yolo/capture_data/2023-08-22/augmented/txt/just_lens/'

# Create main directory
main_directory = '/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/yolo/capture_data/2023-08-22/augmented/dataset'
os.makedirs(main_directory, exist_ok=True)

# Create subdirectories for training, validation, and testing
training_dir = os.path.join(main_directory, 'training')
validation_dir = os.path.join(main_directory, 'validation')
testing_dir = os.path.join(main_directory, 'testing')

for dir in [training_dir, validation_dir, testing_dir]:
    os.makedirs(os.path.join(dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(dir, 'annotations'), exist_ok=True)

# Get list of image and annotation files
image_files = [f for f in os.listdir(image_directory) if f.endswith('.png')]
annotation_files = [f for f in os.listdir(annotation_directory) if f.endswith('.txt')]

# Split into training, validation, and testing
train_files, test_files = train_test_split(image_files, test_size=0.20, random_state=42)
val_files, test_files = train_test_split(test_files, test_size=0.50, random_state=42)

# Function to copy files
def copy_files(files, src_image_dir, src_anno_dir, dest_image_dir, dest_anno_dir):
    for filename in files:
        shutil.copy(os.path.join(src_image_dir, filename), os.path.join(dest_image_dir, filename))
        annotation_file = filename.replace('.png', '.txt')
        shutil.copy(os.path.join(src_anno_dir, annotation_file), os.path.join(dest_anno_dir, annotation_file))

# Copy files to respective directories
copy_files(train_files, image_directory, annotation_directory, os.path.join(training_dir, 'images'), os.path.join(training_dir, 'annotations'))
copy_files(val_files, image_directory, annotation_directory, os.path.join(validation_dir, 'images'), os.path.join(validation_dir, 'annotations'))
copy_files(test_files, image_directory, annotation_directory, os.path.join(testing_dir, 'images'), os.path.join(testing_dir, 'annotations'))

print("Dataset organized successfully!")

Dataset organized successfully!


### 6.4 Use Transfer Learning from the best weights from last training

In [ ]:
from ultralytics import YOLO

# Load the saved model
model = YOLO('best.pt')

# Continue training with the same parameters
results = model.train(data='lens_dataset.yaml', epochs=50, imgsz=640)